In [1]:
import RNA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display
import seaborn as sns

from collections import Counter
from collections import defaultdict
import subprocess
import difflib
import sys
import os
import random
import string
import time

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath



2021-11-19 10:08:50.102844: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-19 10:08:50.102859: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True) # no scientific notation

# import feature_generation
from features import ij_distance, new_move_dist, plt_moves, config_distance, balance_in_all_things, return_shift_moves
from process_features import fp_call, find_moves, process

In [3]:
def adjust_pt(pt, i, j):
    pt_adj = pt.copy()
    if i<0:
        pt_adj[-i] = 0
        pt_adj[-j] = 0
    else:
        pt_adj[i] = j
        pt_adj[j] = i
    return pt_adj

def next_vicinity(lastmove, avail_moves):

    """
    percentage of moves which can continue in the vicinity of the last move
    """
    lasti, lastj = lastmove
    moves = len(avail_moves)
    cntr = 0

    for a in avail_moves:
        i=a[0]
        j=a[1]      
        if lasti + 1 == i or lasti - 1 == i or\
           lastj + 1 == j or lastj - 1 == j:
           cntr += 1
    return cntr

def last_vs_best(lastmove, bestmove, lensequence):
    lasti = abs(lastmove[0])
    lastj = abs(lastmove[1])
    i = abs(bestmove[0])
    j = abs(bestmove[1])
    dist = abs(lasti - i) + abs(lastj - j)
    return dist/(lensequence*2) #, dist, lensequence


def cluster_moves(inputmoves, absolute_values=False):
    """
    cluster moves with DBscan. needs to include the last move as well...
    """

    if absolute_values:
        m = [(abs(i[0]), abs(i[1])) for i in inputmoves]
        moves = np.array(m)
    else:
        m = [(i[0], i[1]) for i in inputmoves]
        moves = np.array(m)

    # print ("clustering", moves)
    clustering = DBSCAN(eps=2, min_samples=1).fit(moves)
    
    # generate clustering dict (move-tuple:label, ...)
    return {i[0]:i[1] for i in zip(m, clustering.labels_)}, clustering.labels_



def structure_evaluation(fc, pt1, pt2, path, lastmove):
    avail_moves = []
    fp_results = []
    en_contrib = []

    available_add = set()
    available_delete = set()
    lensequence = pt1[0]

    # pt1 = list(RNA.ptable(s))
    # pt2 = list(RNA.ptable(s2))
    for pos, (i,j) in enumerate(find_moves(pt1, pt2)):    
        next_en = fc.eval_move_pt(pt1, i, j)
        # mark where we found our move    
        # map energies somehow to [0,1] such that the network will understand.  
        en = np.interp(next_en/100, [-10,10], [0,1]) 
        en_contrib.append(en)
        avail_moves.append((i, j, next_en))

        if i>0:
            available_add.add((i,j))
        elif i<0:
            available_delete.add((i,j))

    if len(avail_moves)==0:
        add_delete = 0
    else:
        add_delete = len(available_delete)/len(avail_moves)

    # print ('best', en_contrib[0], 'avg', en_mean)

    avail_moves.sort(key=lambda x: x[2]) # best move at [0]


    vic = next_vicinity(lastmove, avail_moves)
    vic_best = next_vicinity(lastmove, avail_moves[0:1])

    avail_moves = [(i[0], i[1]) for i in avail_moves]
    avail_moves_abs = [(abs(i[0]), abs(i[1])) for i in avail_moves] # this is sorted as well, best move at [0]

    # absolute_moves+=[lastmove]
    # print ("avail+last", avail_moves)

    # last_index = -1
    # best_index = absolute_moves.index((abs(absolute_moves[0][0]), abs(absolute_moves[0][1])))
    # print ("best", best_index)

    p = [(0, 0)] + [(i[0], i[1]) for i in path]
    clustered_moves, _ = cluster_moves(p, absolute_values=False)
    clustered_moves_abs, _ = cluster_moves(p, absolute_values=True)

    bestmove = avail_moves[0]
    bestmove_abs = avail_moves_abs[0]

    lastmove_abs = abs(lastmove[0]), abs(lastmove[1])

    # last move vicinity is available, but not among the best energy options:

    unique_found = set()
    foundpos = -1
    for pos, m in enumerate(avail_moves):
        l = clustered_moves[m]
        if l == clustered_moves[lastmove]:
            foundpos = len(unique_found) # cant add pos here

        if l not in unique_found:
            unique_found.add(l)
    if foundpos == -1:
        foundpos = len(unique_found)
    foundpos /= len(unique_found)

    unique_found_abs = set()
    foundpos_abs = -1
    for pos, m in enumerate(avail_moves_abs):
        l = clustered_moves_abs[m]
        if l == clustered_moves_abs[lastmove_abs]:
            foundpos_abs = len(unique_found_abs) # cant add pos here

        if l not in unique_found_abs:
            unique_found_abs.add(l)
    if foundpos_abs == -1:
        foundpos_abs = len(unique_found_abs)
    foundpos_abs /= len(unique_found_abs)

    # print (foundpos, last_is_best, last_is_best_abs)
    # print ("bestmove", bestmove, clustered_moves[bestmove], bestmove_abs, clustered_moves_abs[bestmove_abs])
    # print ("lastmove", lastmove, clustered_moves[lastmove], lastmove_abs, clustered_moves_abs[lastmove_abs])

    # distlast = last_vs_best(clustered_moves)

    unique_labels = 1 / len(unique_found)
    # unique_moves = 1/unique_labels

    distlast = last_vs_best(lastmove, bestmove, lensequence)
    unique_moves = 0

    # print ("unique", unique_labels, unique_moves)

    count_available = len(avail_moves)/len(path)


    for pos, (i,j) in enumerate(avail_moves):  
        pt = adjust_pt(pt1, i, j)
        snew = RNA.db_from_ptable(pt)
        # if i > 0:
        #     snew = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
        # if i < 0:
        #     snew = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]

        ptnew = list(RNA.ptable(snew))
        result_new, path = fp_call(sequence, snew, s2, search_width_multiplier)
        fp_results.append(result_new)

    if len(avail_moves) == 0:
        label = 0
        en_mean = 0
        en_std = 0
        best_en = 0

    else:
        label = np.argmin(fp_results)/len(fp_results)

        en_mean = np.mean(en_contrib)
        en_std = np.std(en_contrib)
        en_contrib.sort(key=lambda x: x)
        en_diff = en_contrib[0] / en_mean # best en, relative

        best_en = en_contrib[0]

   
    return label, en_mean, en_std, best_en, foundpos, foundpos_abs, distlast, unique_labels

In [4]:

## todo: drop second best true label after we found the first one 

filename_samples = f'./dataset_100_large.csv'
samples_df = pd.read_csv(filename_samples)

search_width_multiplier = 6
df = pd.DataFrame()
all_results = []



for index, row in samples_df.iterrows():
    # if index != 90:
    #     continue

    
    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2

    bp_dist = RNA.bp_distance(s1, s2)
    fc = RNA.fold_compound(sequence)
    pt1 = list(RNA.ptable(s1))
    pt2 = list(RNA.ptable(s2))

    
    fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=False)
    sE = fp.get_en()/100.0
    path = fp.get_path()

    p = [(0, 0)] + [(i[0], i[1]) for i in path]
    clustered_moves, _ = cluster_moves(p, absolute_values=False)
    clustered_moves_abs, _ = cluster_moves(p, absolute_values=True)

    # print (clustered_moves)

    random_moves = random.choices([i[0] for i in path], k=3)

    ptlast = list(RNA.ptable(s1))
    pt = list(RNA.ptable(s1))

    prev_move_dist = np.zeros((20))  
    lastmove = 0, 0
    all_evals = 0
    results = []

    # find a random structure in the path
    # ignore the last moves, we're not interested in meaningless decisions
    for i, j, en in path:
        if i==0: 
            continue

        prev_move_dist = new_move_dist(abs(lastmove[0]), abs(lastmove[1]), prev_move_dist)
        

        if i in random_moves:
            label, _, _, _, _, _, _, _ = structure_evaluation(fc, pt, pt2, path, lastmove)
                    
            # we take this current intermediate structure and see if we can predict which moves may be the best ones
            # pretend we dont know i and j - substitute with a and b

            avail_moves = []
            ij_moves = []
            found_pos = None
            sElist = []
            balancelist = []

            for pos, (a,b) in enumerate(find_moves(pt, pt2)):    

                ptnew = pt.copy()
                ptnew = adjust_pt(ptnew, a, b)
                s = RNA.db_from_ptable(ptnew)

                fp = findpath.findpath_single(sequence, s, s2, search_width_multiplier=search_width_multiplier, mp=False)
                sE = fp.get_en()/100.0
                sElist.append(sE)


                next_en = fc.eval_move_pt(pt, a, b)
                avail_moves.append((a, b, next_en))

            avail_moves.sort(key=lambda x: x[2])
            en_list = np.array([[x[2] for x in avail_moves]])
            en_list_scaled = min_max_scaler.fit_transform(en_list.T).T[0]

            # print (index, i, label, lastmove)
            # print (RNA.db_from_ptable(pt), prev_move_dist, i, j)
            # print (avail_moves)

            i_values = []
            j_values = []

            for pos, (a, b, en) in enumerate(avail_moves):
            
                ptnew = pt.copy()
                ptnew = adjust_pt(ptnew, a, b)
                s = RNA.db_from_ptable(ptnew)

                fp = findpath.findpath_single(sequence, s, s2, search_width_multiplier=search_width_multiplier, mp=False)
                sE = fp.get_en()/100.0
                sElist.append(sE)

                # i_bar = int(round(abs(a) / 5.0))
                # j_bar = int(round(abs(b) / 5.0))
                i_bar = int(abs(a) / 5.001)
                j_bar = int(abs(b) / 5.001)

                # path = fp.get_path()
                i_value = round(prev_move_dist[i_bar], 4)
                j_value = round(prev_move_dist[j_bar], 4)
                i_values.append(i_value)
                j_values.append(j_value)

                balancelist.append(balance_in_all_things(s1, s2, s) / len(path))
                                

            sElist = min_max_scaler.fit_transform(np.array([sElist]).T).T[0]
            balancelist = min_max_scaler.fit_transform(np.array([balancelist]).T).T[0]

            for (a,b, _), en, sEscaled, i_value, j_value, balance in zip(avail_moves, en_list_scaled, sElist, i_values, j_values, balancelist):                
                # how far are we away from the pred. label?
                labeldiff = abs(label-en)

                if a<0:
                    insert_or_delete = 0
                else:
                    insert_or_delete = 1

                helix_extension = config_distance(pt, (a,b))


                # print (a, b, en, labeldiff, i_value, j_value, sE)
                label = sEscaled
                results.append((sequence, RNA.db_from_ptable(pt), s2, a, b, label, en, labeldiff, i_value, j_value, insert_or_delete, balance, helix_extension))

            # results.append((sequence, RNA.db_from_ptable(pt), s2, i, j, label, en_mean, en_std, best_en, foundpos, foundpos_abs, distlast, unique_labels))

            all_evals += label       
            # print (i, eval)  

        # move on along the predefined path
        pt = adjust_pt(ptlast, i, j)
        lastmove = i, j
        ptlast = pt

    if all_evals != 0:
        all_results += results


    # break

    if index%10 == 0:        
        print (index, end=" ")
        # print (all_results)
        df = pd.DataFrame(all_results, columns=['sequence', 's', 's2', 'i', 'j', 'target', 'en', 'labeldiff', 'i_value', 'j_value', 'insert_or_delete', 'balance', 'helix_extension'])# .set_index('sequence')
        filename = f'./dataset_100_large_train.csv'
        df.to_csv(filename)

    # break
    if index == 100:
        break


print (df.head())



0 

KeyboardInterrupt: 

In [77]:
# filename = f'./dataset_100_large_train.csv'
# df.to_csv(filename)
df

,sequence,s,s2,i,j,target,en,labeldiff,i_value,j_value,insert_or_delete,balance,helix_extension
0,CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGA...,(.........(((((((((.(.(((.((((...(((((....(((....,(((((((((((((.......))))(((......))).(((....((...,2,92,0.000000,0.000000,0.000000e+00,0.4815,0.4810,1,0.0,1
1,CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGA...,(.........(((((((((.(.(((.((((...(((((....(((....,(((((((((((((.......))))(((......))).(((....((...,-21,-80,0.000000,0.384615,3.846154e-01,0.0008,0.0000,0,1.0,0
2,CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGA...,(.........(((((((((.(.(((.((((...(((((....(((....,(((((((((((((.......))))(((......))).(((....((...,-15,-86,0.000000,0.406593,4.065934e-01,0.2213,0.0558,0,1.0,0
3,CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGA...,(.........(((((((((.(.(((.((((...(((((....(((....,(((((((((((((.......))))(((......))).(((....((...,-43,-58,0.576923,0.516484,5.164835e-01,0.0000,0.0000,0,1.0,1
4,CUGACUCCGGGUGCCCCGUACACCGAGACCAAGCUCACCCUCCGGA...,(.........(((((((((.(.(((.((((...(((((....(((....,(((((((((((((.......))))(((......))).(((....((...,-25,-77,0.000000,0.527473,4.945055e-02,0.0008,0.0000,0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAG...,.(((...((.(.(.((((((....)))))).).)...))......)...,.(((.((((..(..((((((....)))))).).........)))))...,-13,-32,0.705882,0.459459,2.829889e-01,0.0814,0.0004,0,1.0,0
373,CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAG...,.(((...((.(.(.((((((....)))))).).)...))......)...,.(((.((((..(..((((((....)))))).).........)))))...,7,44,0.000000,0.486486,2.193959e-01,0.5275,0.1142,1,0.0,0
374,CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAG...,.(((...((.(.(.((((((....)))))).).)...))......)...,.(((.((((..(..((((((....)))))).).........)))))...,6,45,0.000000,0.729730,7.297297e-01,0.5275,0.1142,1,0.0,0
375,CCUCUUGGCAAGGAACUGUUGUAUGACGGUACGUAACGCGCGCCAG...,.(((...((.(.(.((((((....)))))).).)...))......)...,.(((.((((..(..((((((....)))))).).........)))))...,-9,-38,1.000000,0.918919,9.189189e-01,0.5275,0.0115,0,1.0,1


## load saved model, compare prediction with true labels

In [9]:
reloaded_model = tf.keras.models.load_model('dataset_model')

data = {'en': [0.395604],
        'labeldiff': 1.456044e-01,
        'i_value': 0.4899,
        'j_value': 0.2500,
        'insert_or_delete': 0,
        'balance': 1.0,
        'helix_extension': 1
        }

test_features = pd.DataFrame.from_dict(data)

y = reloaded_model.predict(test_features)[0][0]
y




0.08048675

In [43]:
filename_samples = f'./dataset_100_large.csv'
samples_df = pd.read_csv(filename_samples)

search_width_multiplier = 6
df = pd.DataFrame()
all_results = []



for index, row in samples_df.iterrows():
    if index != 417:
        continue

    
    sequence = row.sequence
    s1 = row.s1
    s2 = row.s2

    bp_dist = RNA.bp_distance(s1, s2)
    fc = RNA.fold_compound(sequence)
    pt1 = list(RNA.ptable(s1))
    pt2 = list(RNA.ptable(s2))

    
    fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)
    sE = fp.get_en()/100.0
    path = fp.get_path()

    p = [(0, 0)] + [(i[0], i[1]) for i in path]
    clustered_moves, _ = cluster_moves(p, absolute_values=False)
    clustered_moves_abs, _ = cluster_moves(p, absolute_values=True)

    # print (clustered_moves)

    random_moves = random.choices([i[0] for i in path], k=3)
    random_moves = [50, -51, -49, 12]
    random_moves = [-49]

    ptlast = list(RNA.ptable(s1))
    pt = list(RNA.ptable(s1))

    prev_move_dist = np.zeros((20))  
    lastmove = 0, 0
    all_evals = 0
    results = []

    # find a random structure in the path
    # ignore the last moves, we're not interested in meaningless decisions
    for i, j, en in path:
        if i==0: 
            continue

        prev_move_dist = new_move_dist(abs(lastmove[0]), abs(lastmove[1]), prev_move_dist)
        

        if i in random_moves:
        # if True:
            label, _, _, _, _, _, _, _ = structure_evaluation(fc, pt, pt2, path, lastmove)
            
            current_en = fc.eval_structure_pt(pt)/100
            print (f' {RNA.db_from_ptable(pt)[0:100]} {current_en:5} label: {label:6.3}, move: {(i,j)}')

            # we take this current intermediate structure and see if we can predict which moves may be the best ones
            # pretend we dont know i and j - substitute with a and b

            avail_moves = []
            ij_moves = []
            found_pos = None
            sElist = []
            balancelist = []

            for pos, (a,b) in enumerate(find_moves(pt, pt2)):    
                next_en = fc.eval_move_pt(pt, a, b)
                avail_moves.append((a, b, next_en))

            avail_moves.sort(key=lambda x: x[2])
            en_list = np.array([[x[2] for x in avail_moves]])
            en_list_scaled = min_max_scaler.fit_transform(en_list.T).T[0]

            print (index, i, label, lastmove)
            # print (RNA.db_from_ptable(pt), prev_move_dist, i, j)
            # print (avail_moves)

            i_values = []
            j_values = []

            for pos, (a, b, en) in enumerate(avail_moves):
            
                ptnew = pt.copy()
                ptnew = adjust_pt(ptnew, a, b)
                s = RNA.db_from_ptable(ptnew)

                fp = findpath.findpath_single(sequence, s, s2, search_width_multiplier=search_width_multiplier, mp=False)
                sE = fp.get_en()/100.0
                sElist.append(sE)

                # i_bar = int(round(abs(a) / 5.0))
                # j_bar = int(round(abs(b) / 5.0))
                i_bar = int(abs(a) / 5.001)
                j_bar = int(abs(b) / 5.001)

                # path = fp.get_path()
                i_value = round(prev_move_dist[i_bar], 4)
                j_value = round(prev_move_dist[j_bar], 4)
                i_values.append(i_value)
                j_values.append(j_value)

                balancelist.append(balance_in_all_things(s1, s2, s) / len(path))
                                

            sElist = min_max_scaler.fit_transform(np.array([sElist]).T).T[0]
            balancelist = min_max_scaler.fit_transform(np.array([balancelist]).T).T[0]

            for (a,b, _), en, sEscaled, i_value, j_value, balance in zip(avail_moves, en_list_scaled, sElist, i_values, j_values, balancelist):                
                # how far are we away from the pred. label?
                labeldiff = abs(label-en)

                if a<0:
                    insert_or_delete = 0
                else:
                    insert_or_delete = 1

                helix_extension = config_distance(pt, (a,b))


                # print (a, b, en, labeldiff, i_value, j_value, sE)
                label = sEscaled

                # predit label
                data = {'en': [en],
                        'labeldiff': labeldiff,
                        'i_value': i_value,
                        'j_value': j_value,
                        'insert_or_delete': insert_or_delete,
                        'balance': balance,
                        'helix_extension': helix_extension
                        }
                test_features = pd.DataFrame.from_dict(data)
                pred_label = reloaded_model.predict(test_features)[0][0]

                
                # results.append((sequence, RNA.db_from_ptable(pt), s2, a, b, label, pred_label, en, labeldiff, i_value, j_value, insert_or_delete, balance, helix_extension))
                results.append((sequence, i, j, a, b, label, pred_label, en, labeldiff, i_value, j_value, insert_or_delete, balance, helix_extension))

            # results.append((sequence, RNA.db_from_ptable(pt), s2, i, j, label, en_mean, en_std, best_en, foundpos, foundpos_abs, distlast, unique_labels))

            all_evals += label       
            # print (i, eval)  

        # move on along the predefined path
        pt = adjust_pt(ptlast, i, j)
        lastmove = i, j
        ptlast = pt

    if all_evals != 0:
        all_results += results


    # break

    # if index%10 == 0:        
        # print (index, end=" ")
        # print (all_results)
    df = pd.DataFrame(all_results, columns=['sequence', 's', 's2', 'i', 'j', 'target', 'pred_label', 'en', 'labeldiff', 'i_value', 'j_value', 'insert_or_delete', 'balance', 'helix_extension'])# .set_index('sequence')
        # filename = f'./dataset_100_large_train.csv'
        # df.to_csv(filename)

    # break
    if index == 100:
        break

df

 ..((((.......((.((((.......)))))).......))))((((((.(((...))).)....)))))....((..(((.((.....)).)))..)) -20.3 label:    0.2, move: (-49, -67)
417 -49 0.2 (54, 58)


,sequence,s,s2,i,j,target,pred_label,en,labeldiff,i_value,j_value,insert_or_delete,balance,helix_extension
0,AACGGGGGCUUCAACUCGCUCAGAAUCAGCGGUAUAGAUAUCCGGG...,-49,-67,-14,-33,0.322581,0.173361,0.000000,0.200000,0.0003,0.0001,0,1.0,1
1,AACGGGGGCUUCAACUCGCUCAGAAUCAGCGGUAUAGAUAUCCGGG...,-49,-67,-49,-67,0.000000,0.123355,0.000000,0.322581,0.0555,0.1631,0,1.0,1
2,AACGGGGGCUUCAACUCGCUCAGAAUCAGCGGUAUAGAUAUCCGGG...,-49,-67,12,37,0.419355,0.332342,0.419355,0.419355,0.0003,0.0003,1,0.0,0
3,AACGGGGGCUUCAACUCGCUCAGAAUCAGCGGUAUAGAUAUCCGGG...,-49,-67,11,38,0.870968,0.624632,0.870968,0.451613,0.0003,0.0003,1,0.0,0
4,AACGGGGGCUUCAACUCGCUCAGAAUCAGCGGUAUAGAUAUCCGGG...,-49,-67,-15,-32,1.000000,1.002777,1.000000,0.129032,0.0003,0.0001,0,1.0,1
